In [1]:
#There are several sources of Pokemon data. 
#After clicking around the JSON returned from the PokeAPI in this cell, I decided to pursue another method

# import requests

# pokemonName = "ditto"
# api_url = f"https://pokeapi.co/api/v2/pokemon/{pokemonName}"
# response = requests.get(api_url)
# response.json()

In [2]:
#import libraries
import requests, six
import lxml.html as lh
import pandas as pd


In [3]:
#get data from website
url='http://pokemondb.net/pokedex/all'
page = requests.get(url)

#store data as doc
doc = lh.fromstring(page.content)

#data is stored after <tr, so we need to parse first row for header
tr_elements = doc.xpath('//tr')


In [4]:

#Create empty list to store each element
col=[]
i=0

#Take first element from each row to create header
for j in tr_elements[0]:
    i+=1
    name=j.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
#Store remaining data beginning on the second row
for k in range(1,len(tr_elements)):
    currentRow=tr_elements[k]
    
    #If row length !== 10, the //tr data is not from our table 
    if len(currentRow)!=10:
        break
    
    i=0
    
    #Iterate through each element of the row
    for l in currentRow.iterchildren():
        data=l.text_content() 
        if i>0:
        #Convert any numbers to ints
            try:
                data=int(data)
            except:
                pass
        #Append the data
        col[i][1].append(data)
        #Increment i to move the index next
        i+=1

1:"#"
2:"Name"
3:"Type"
4:"Total"
5:"HP"
6:"Attack"
7:"Defense"
8:"Sp. Atk"
9:"Sp. Def"
10:"Speed"


In [5]:
#Create dataframe to use pandas, and print to debug

Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0001,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,0002,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,0003,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,0003,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,0004,Charmander,Fire,309,39,52,43,60,50,65


In [6]:
#Reformat data to be prettier
#Names should have parantheses around second part
def addParantheses(word):
    list = [x for x in word]
    for charIndex in range(1, len(list)):
        if list[charIndex].isupper():
            list[charIndex] = ' ' + list[charIndex]
    finalList = ''.join(list).split(' ')
    length = len(finalList)
    if length>1:
        finalList.insert(1,'(')
        finalList.append(')')
    return ' '.join(finalList)



In [7]:
#Types should be separated 
def breakOnUpper(word):
    list = [x for x in word]
    for charIndex in range(1, len(list)):
        if list[charIndex].isupper():
            list[charIndex] = '-' + list[charIndex]
    finalList = ''.join(list).split('-')
    return finalList


In [8]:
df['Name']=df['Name'].apply(addParantheses)
df.head()

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0001,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,0002,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,0003,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,0003,Venusaur ( Mega Venusaur ),Grass Poison,625,80,100,123,122,120,80
4,0004,Charmander,Fire,309,39,52,43,60,50,65


In [9]:
df['Type']=df['Type'].apply(breakOnUpper)
df.head()

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0001,Bulbasaur,"[Grass , Poison]",318,45,49,49,65,65,45
1,0002,Ivysaur,"[Grass , Poison]",405,60,62,63,80,80,60
2,0003,Venusaur,"[Grass , Poison]",525,80,82,83,100,100,80
3,0003,Venusaur ( Mega Venusaur ),"[Grass , Poison]",625,80,100,123,122,120,80
4,0004,Charmander,[Fire ],309,39,52,43,60,50,65


In [10]:
#Save dataframe to json to use in pandas
df.to_json('PokemonData.json')

In [11]:
#Make list of pokemon fun facts to add some spice to the PokeDex
#Get best and worst pokemon for each of the stats

def max_stats(df, columns):
    messages = []
    message = ''
    for stat in columns:
        statValue = df[stat].max()
        name = df[df[stat]==df[stat].max()]['Name'].values[0]
        message = name+' has the greatest '+stat+' of '+str(statValue)+'.'
        messages.append(message)
    return messages

def min_stats(df, columns):
    messages = []
    message = ''
    for stat in columns:
        statValue = df[stat].min()
        name = df[df[stat]==df[stat].min()]['Name'].values[0]
        message = name+' has the worst '+stat+' of '+str(statValue)+'.'
        messages.append(message)
    return messages

In [12]:
stats=['HP', 'Attack', 'Defense', 'Sp. Atk','Sp. Def','Speed','Total']
max_stats = max_stats(df, stats)
print(max_stats)

min_stats = min_stats(df,stats)
print(min_stats)

['Blissey has the greatest HP of 255.', 'Mewtwo (  Mega  Mewtwo  X ) has the greatest Attack of 190.', 'Eternatus (  Eternamax ) has the greatest Defense of 250.', 'Mewtwo (  Mega  Mewtwo  Y ) has the greatest Sp. Atk of 194.', 'Eternatus (  Eternamax ) has the greatest Sp. Def of 250.', 'Regieleki has the greatest Speed of 200.', 'Eternatus (  Eternamax ) has the greatest Total of 1125.']
['Shedinja has the worst HP of 1.', 'Chansey has the worst Attack of 5.', 'Chansey has the worst Defense of 5.', 'Sandshrew (  Alolan  Sandshrew ) has the worst Sp. Atk of 10.', 'Caterpie has the worst Sp. Def of 20.', 'Shuckle has the worst Speed of 5.', 'Wishiwashi (  Solo  Form ) has the worst Total of 175.']


In [13]:
#Create GUI

In [14]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [16]:
import ipywidgets as widgets
from ipywidgets import interact
#Spend a long time finding out that Image from ipywidgets has a known bug that prevents it from loading url images
#Bug not present in IPython display
from IPython.display import Image
import numpy as np
import random


# Create the main window
title = widgets.Label('PokeDex', style=dict(
    font_weight='bold',
    font_variant="small-caps",
    text_color='black',
    text_decoration='underline',
    font_size='large'
))

# Create the dropdown menu
pokemon_names = df["Name"].to_list()
# pokemon_name_dropdown = widgets.Dropdown(options=pokemon_names)


#Dropdown was too long.
#Create Combobox to search for Pokemon name
choosePokemon = widgets.Combobox(
    value='Charmander',
    placeholder='Type in Pokemon Name',
    options=pokemon_names,
    description='Pokemon: ',
    ensure_option=True,
    disabled=False
)

#Display Pokemon stats after selection
def show_stats(pokemonName):
    index = pokemon_names.index(pokemonName)
    val= pokemonName
    cond = df['Name']== val
        
    #Image
    base = 'http://img.pokemondb.net/artwork/'
    img_url = base + val.lower() + '.jpg'
    image = Image(url=img_url)

    
        
    #Set values
    name = 'Name:\t\t\t'+val+'\n\n'
    ty = 'Type:\t\t\t'+ ', '.join(df[cond]['Type'].values[0])+'\n\n'
    hp = 'HP:\t\t\t'+ str(df[cond]['HP'].values[0])+'\n\n'
    atk = 'Attack:\t\t\t'+str(df[cond]['Attack'].values[0])+'\n\n'
    satk = 'Sp. Attack:\t\t'+str(df[cond]['Sp. Atk'].values[0])+'\n\n'
    deff = 'Defense:\t\t\t'+str(df[cond]['Defense'].values[0])+'\n\n'
    sdef = 'Sp. Defense:\t\t'+str(df[cond]['Sp. Def'].values[0])+'\n\n'
    speed = 'Speed:\t\t\t'+str(df[cond]['Speed'].values[0])+'\n\n'
    total = 'Total:\t\t\t'+str(df[cond]['Total'].values[0])+'\n\n'
        
    #Add text
    final = name+ty+hp+atk+satk+deff+sdef+speed+total
    display(widgets.Label(final, style=dict(
    font_weight='bold',
    font_variant="small-caps",
    text_color='black',
    font_size='large'
    )))
    display(image)
    
    

#Random fact button
randomFact = widgets.Button(description='Random Stat Fact')
randomFact.style.button_color = 'lightgreen'
randomFact.style.font_variant = 'small-caps'

def on_button_clicked(b):
    allFacts = np.concatenate((max_stats, min_stats))
    factNumber = random.randint(0, len(allFacts))
    fact = allFacts[factNumber]
    with output:
        print(fact)

randomFact.on_click(on_button_clicked)
output = widgets.Output()


# Layout the widgets
display(title)
display(randomFact, output)
interact(show_stats, pokemonName=choosePokemon)



Label(value='PokeDex', style=LabelStyle(font_size='large', font_variant='small-caps', font_weight='bold', text…

Button(description='Random Stat Fact', style=ButtonStyle(button_color='lightgreen', font_variant='small-caps')…

Output()

interactive(children=(Combobox(value='Charmander', description='Pokemon: ', ensure_option=True, options=('Bulb…

<function __main__.show_stats(pokemonName)>